In [1]:
import datetime
import time
import pytz

import pandas as pd
import numpy as np
import pandas_market_calendars as pmc

import sys, os
sys.path.append(os.path.join(os.path.dirname('.'), '..', 'lib'))
sys.path.insert(0, '/Users/chris/programming/interactivebrokers')
import ibapi

sys.path.insert(0, '/Users/chris/programming/analyses/trading')
import tradingutils as tutils

MAX_WAIT_TIME = 90

In [2]:
import master
import marketdata
import contracts
import helper

PORT = 7497
md_app = marketdata.get_instance(port=PORT)
ct_app = contracts.get_instance(port=PORT)

Attempting to connect with unused clientId...


ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
serverVersion:%s connectionTime:%s:clientId:%s
MarketDataApp connecting to IB...
Attempting to connect with unused clientId...


ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
ERROR: request -1. (Unimplemented error handler).
serverVersion:%s connectionTime:%s:clientId:%s
MarketDataApp connecting to IB...


In [3]:
def download_and_save_daily_data(md_app, contract, start_date, end_date, data_type):
    period_start = start_date
    while period_start < end_date:
        assert isinstance(period_start, datetime.date), 'Start/end dates must be dates - not datetimes.'
        
        # Create a data request object
        reqObj = create_daily_data_request(contract, frequency=frequency, data_type=data_type, date=period_start)
        
        # Download the data, if the market was open on this day
        if reqObj is not None:
            df = download_daily_data(reqObj)

            # Save to file    
            tws_date = datetime.datetime.strftime(period_start, '%Y%m%d')
            filename = tutils.save_daily_ib_data(df, ticker, frequency=reqObj.frequency, 
                                                 tws_date=tws_date, data_type=data_type)
            print('Saved data to {}'.format(filename))

        # Move to the next start date
        period_start += datetime.timedelta(days=1)
    
def create_daily_data_request(contract, frequency, data_type, date):
    """Create a data request object to get all daily data at a certain frequency.
    """
    datetime_intervals, _ = get_trading_intervals(date)
    if not datetime_intervals:
        # Skip days with no trading hours
        return None
    else:
        # Only request data between start/end of trading hours
        start_time = datetime_intervals[0][0]
        end_time = datetime_intervals[-1][1]

        # Get the request object
        tws_start = helper.convert_datetime_to_tws_date(start_time)
        tws_end = helper.convert_datetime_to_tws_date(end_time)    
        reqObjList = md_app.create_historical_data_request([contract], is_snapshot=True, frequency=frequency, 
                                                use_rth=False, data_type=data_type,
                                                start=tws_start, end=tws_end, duration="")
        reqObj = reqObjList[0]
        assert reqObj is not None, 'Request parameters are invalid.'
        return reqObj
    
def download_daily_data(reqObj):
    reqObj.place_request()    
    return retrieve_requested_data(reqObj)

def _get_number_of_expected_results(reqObj):
    """ Get the number of expected results from each subrequest.
    """
    sub_requests = reqObj.get_subrequests()
    n_requests = len(sub_requests)
    n_expected_results = [None] * n_requests

    freq_helper = helper.TimeHelper(reqObj.frequency, 'frequency')
    total_seconds_per_bar = freq_helper.total_seconds()
    for j, sub_req in enumerate(sub_requests):
        total_seconds_in_request = (sub_req.get_end_tws() - sub_req.get_start_tws()).total_seconds()
        n_expected_results[j] = int(total_seconds_in_request / total_seconds_per_bar)
    assert all([x > 0 for x in n_expected_results]), \
                            'All requests should expect a positive number of results.'
    return n_expected_results

def retrieve_requested_data(reqObj):
    # Wait until the data has been returned by IB
    data = reqObj.get_data()
    n_requests = len(reqObj.get_subrequests())
    n_expected_results = _get_number_of_expected_results(reqObj)
    
    def calc_count(d): 
        ct = [None] * n_requests
        if d:
            for idx in range(n_requests):
                ct[idx] = len(d[idx]) == n_expected_results[idx]
        return ct
        
    # Wait until all datasets have been returned
    t0 = time.time()
    ct = calc_count(data)
    while sum(ct) != n_requests and time.time() - t0 < MAX_WAIT_TIME:
        print('Waiting for results {}: {}/{}'.format(int(time.time() - t0), sum(ct), n_requests))
        time.sleep(5)
        ct = calc_count(data)

    if sum(ct) != n_requests:
        # If some requests were not successful, we need to re-run them
        print("Restarting unsuccessful request(s)...")
        sub_requests = reqObj.get_subrequests()
        for j in range(n_requests):
            if len(data[j]) < n_expected_results[j]:
                sub_requests[j].reset()
                sub_requests[j].place_request()
                
        return retrieve_requested_data(reqObj)
    else:
        print('Finished downloading for {}. Time elapsed: {}'.format(reqObj.start, time.time() - t0))
        return reqObj.get_dataframe()

def download_all_requests_in_list(reqObjList):
    for j, reqObj in enumerate(reqObjList):
        print('Processing request {} for {} {} from {} to {}...'.format(j, reqObj.contract.localSymbol, 
                                                reqObj.data_type, reqObj.start, reqObj.end))

        success = False
        t0 = time.time()
        while not success:
            # Place the time series request
            reqObj.place_request()

            # Wait until the data has been returned by IB
            data = retrieve_requested_data(reqObj)

        print('Finished. Time elapsed: {}'.format(time.time() - t0))

        # Extract and format data
        df = reqObj.get_dataframe()

        # Save to file
        tws_date = reqObj.start.split(' ')[0]
        symbol = reqObj.contract.localSymbol
        filename = tutils.save_daily_ib_data(df, ticker, frequency=reqObj.frequency, 
                                             tws_date=tws_date, data_type=reqObj.data_type)
        print(filename)
        
def get_trading_intervals(date):    
    # Create a datetime representing midnight on the target date
    dt = datetime.datetime.combine(date, datetime.datetime.min.time())
    
    if dt.weekday() < 4:
        # Monday to Thursday trading is open all day except
        #     closing from 15:15 - 15:30 CT and 16:00 - 17:00 CT for scheduled maintenance
        trading_intervals = [(dt, dt + datetime.timedelta(minutes=16*60 + 15)),
                             (dt + datetime.timedelta(minutes=16*60 + 30), dt + datetime.timedelta(hours=17)),
                             (dt + datetime.timedelta(hours=18), dt + datetime.timedelta(hours=24))]
    elif dt.weekday() == 4:
        # Friday trading is open until 16:00 CT
        trading_intervals = [(dt, dt + datetime.timedelta(minutes=16*60 + 15)),
                             (dt + datetime.timedelta(minutes=16*60 + 30), dt + datetime.timedelta(hours=17))]
        
    elif dt.weekday() == 5:
        # Market is closed on Saturdays    
        trading_intervals = []
    elif dt.weekday() == 6:
        # Sunday trading opens at 5 pm CT/6 pm EST
        trading_intervals = [(dt + datetime.timedelta(hours=18), dt + datetime.timedelta(hours=24))]

    # Specify that all of these dates are in EST time zone
    tzone = pytz.timezone(helper.TIMEZONE_EST)
    datetime_intervals = [None] * len(trading_intervals)
    timestamp_intervals = [None] * len(trading_intervals)
    for j in range(len(trading_intervals)):
        start = tzone.localize(trading_intervals[j][0])
        end = tzone.localize(trading_intervals[j][1])
        datetime_intervals[j] = (start, end) 
        timestamp_intervals[j] = (helper.get_utc_timestamp_from_datetime(start),
                                 helper.get_utc_timestamp_from_datetime(end))   

    return datetime_intervals, timestamp_intervals

def is_data_within_trading_hours(df, date):
    _, timestamp_intervals = get_trading_intervals(date)
    
    if not timestamp_intervals:
        return df.shape[0] == 0
    else:
        if df.index.values[0] < timestamp_intervals[0][0]:
            return False
        if df.index.values[-1] > timestamp_intervals[-1][1]:
            return False        

        # Check that all data points are within a valid interval
        valid = np.zeros_like(df.index.values, dtype=bool)
        for interval in timestamp_intervals:
            start, end = interval
            is_in_interval = (start <= df.index.values) & (df.index.values <= end)
            valid = valid | is_in_interval
            
        return np.all(valid)      
    
def find_gaps_in_data(df, date, min_gap=300):
    utc_datetimes = [tutils.convert_tws_date_to_utc_datetime(d) for d in df.date.values]
    utc_timestamps = [dt.timestamp() for dt in utc_datetimes]
    df.index = pd.Index(utc_timestamps, name='utc_timestamp')

    _, timestamp_intervals = get_trading_intervals(date)

    if not timestamp_intervals:
        print('{} should be empty'.format(date))
        return []
    elif not is_data_within_trading_hours(df, date):
        print('{}: Some data occurs outside of trading hours.'.format(date))
        return []
    else:
        # Add times outside of trading hours to the index values so that we can 
        #    more easily gauge where there is missing data
        index_vals = np.sort(df.index.values)
        start_day = timestamp_intervals[0][0]
        end_day = timestamp_intervals[-1][1]
        if index_vals[0] != start_day:
            index_vals = np.hstack([start_day, index_vals])
        if index_vals[-1] != end_day:
            index_vals = np.hstack([index_vals, end_day])
        for j in range(1, len(timestamp_intervals)):
            close_time = timestamp_intervals[j-1][1]
            open_time = timestamp_intervals[j][0]
            nontrading_times = np.arange(close_time, open_time)
            index_vals = np.sort(np.hstack([index_vals, nontrading_times]))

        diff = index_vals[1:] - index_vals[:-1]
        gaps = []
        for idx in np.where(diff > min_gap)[0]:
            gaps.append((diff[idx],
                         idx+1, 
                         helper.convert_utc_timestamp_to_datetime(index_vals[idx], tz_name=helper.TIMEZONE_EST),
                         helper.convert_utc_timestamp_to_datetime(index_vals[idx+1], tz_name=helper.TIMEZONE_EST),
                        ))    
        return gaps    
    
def get_exchange_trading_hours(exchange, start_date, end_date):
    calendar = pmc.get_calendar(exchange)
    utc_trading_hours = calendar.schedule(start_date=start_date, end_date=end_date)

    def convert_utc_datetime_to_est(utc_hours):
        datetimes = pd.DatetimeIndex(utc_hours.values)
        tz_est = pytz.timezone(helper.TIMEZONE_EST)
        est_hours = []
        for utc_dt in datetimes:
            est_hours.append(pytz.utc.localize(utc_dt).astimezone(tz_est))
        return est_hours

    est_trading_hours = utc_trading_hours.copy()
    open_hours = convert_utc_datetime_to_est(utc_trading_hours.market_open)
    est_trading_hours.market_open = [d - datetime.timedelta(minutes=1) for d in open_hours]
    est_trading_hours.market_close = convert_utc_datetime_to_est(utc_trading_hours.market_close)
    return est_trading_hours    


In [ ]:
partial_contract = ibapi.contract.Contract()
partial_contract.symbol = 'VIX'
partial_contract.secType = 'FUT'
partial_contract.lastTradeDateOrContractMonth = '202005'
partial_contract.currency = 'USD'

#ct_app.match_contract(partial_contract)
matching_contracts = ct_app.get_all_matching_contracts(partial_contract)
matching_contracts

#contract = matching_contracts[0]
#ct_app.add_to_saved_contracts([contract])
#ct_app.get_contract(contract.localSymbol)

In [4]:
ticker = 'ESM0'
frequency = '1s'
data_types = ['TRADES']

start = datetime.date(2020, 3, 24)
end = datetime.date(2020, 3, 28)

contract = ct_app.get_contract(ticker)
for data_type in data_types:
    download_and_save_daily_data(md_app, contract, start, end, data_type=data_type)

Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Waiting for results 0: 46/48
Finished downloading for 20200324 00:00:00. Time elapsed: 5.000666856765747
Saved data to data/ESM0_1s_20200324_TRADES.csv
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 556.9834660053253 seconds to avoid pacing violation on total requests.


ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfuture.nj


ERROR: request -1
ERROR: request -1


ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture.nj
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture.nj
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture.nj
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture.nj


ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Waiting for results 0: 47/48
Finished downloading for 20200325 00:00:00. Time elapsed: 5.005151987075806
Saved data to data/ESM0_1s_20200325_TRADES.csv
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 555.3613

In [36]:
# Create a time series request
ticker = 'ESM0'
frequency = '1s'
data_type = 'ASK'
start = '20200315 00:00:00'
end = '20200316 00:00:00'
duration = ''

contractList = [ct_app.get_contract(ticker)]
reqObjList = md_app.create_historical_data_request(contractList, is_snapshot=True, frequency=frequency, 
                                        use_rth=False, data_type=data_type,
                                        start=start, end=end, duration=duration)

In [37]:
# Place the time series request
reqObj = reqObjList[0]
reqObj.place_request()

Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...


In [39]:
df = retrieve_requested_data(reqObj)
tutils.set_timestamp_index(df)

Finished downloading for 20200315 00:00:00. Time elapsed: 2.002716064453125e-05


In [19]:
tws_date = start.split(' ')[0]
filename = tutils.save_daily_ib_data(df, reqObj.contract.localSymbol, frequency=reqObj.frequency, 
                                     tws_date=tws_date, data_type=reqObj.data_type)
print('Saved to {}'.format(filename))

Saved to data/ESM0_1M_20200315_TRADES.csv


In [13]:
ticker = 'ESM0'
start = datetime.datetime(2020, 3, 15)
end = datetime.datetime(2020, 4, 20)
data_type = 'ASK'

frequency = '1s'
files = tutils._get_valid_files(ticker, frequency, start, end, data_type)
data_frames_1s = dict()
for f in files:
    date = f.split('_')[-2]
    df = pd.read_csv(f)
    df.set_index('utc_timestamp', inplace=True)
    data_frames_1s[date] = df
    
frequency = '1M'
files = tutils._get_valid_files(ticker, frequency, start, end, data_type)
data_frames_1M = dict()
for f in files:
    date = f.split('_')[-2]
    df = pd.read_csv(f)
    df.set_index('utc_timestamp', inplace=True)
    data_frames_1M[date] = df    

In [354]:
date = start
status = dict()
t0 = time.time()
while date <= min(end, datetime.datetime.now()):
    tws_date = datetime.datetime.strftime(date, '%Y%m%d')
    
    datetime_intervals, timestamp_intervals = get_trading_intervals(date)
    if not timestamp_intervals:
        pass
    elif tws_date not in data_frames_1s:
        status[date] = ['Missing seconds']
    elif tws_date not in data_frames_1M:
        status[date] = ['Missing minutes']
    else:
        df_S = data_frames_1s[tws_date]
        df_M = data_frames_1M[tws_date]

        index_vals_S = np.floor(df_S.index.values / 60) * 60
        index_vals_S = set(index_vals_S.astype(int))

        if data_type == 'TRADES':
            index_vals_M = set(df_M.iloc[df_M.volume.values > 0].index.astype(int).values)
        else:
            index_vals_M = set(df_M.index.astype(int).values)

        tz_est = pytz.timezone(helper.TIMEZONE_EST)
        status[tws_date] = [helper.convert_utc_timestamp_to_datetime(d).astimezone(tz_est) for d \
                                             in sorted(index_vals_M - index_vals_S)]        
    # Move to the next date
    date = date + datetime.timedelta(days=1)

In [355]:
status

{'20200315': [],
 '20200316': [],
 '20200317': [],
 '20200318': [],
 '20200319': [],
 '20200320': [],
 '20200322': [],
 '20200323': [],
 '20200324': [],
 '20200325': [],
 '20200326': [],
 '20200327': [],
 '20200329': [],
 '20200330': [],
 '20200331': [],
 '20200401': [],
 '20200402': [],
 '20200403': [],
 '20200405': [],
 '20200406': [],
 '20200407': [],
 '20200408': [],
 '20200409': [],
 datetime.datetime(2020, 4, 10, 0, 0): ['Missing seconds'],
 '20200412': [],
 '20200413': [],
 '20200414': [],
 '20200415': [],
 '20200416': [],
 '20200417': [],
 '20200419': [],
 datetime.datetime(2020, 4, 20, 0, 0): ['Missing seconds']}

ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfuture.nj


ERROR: request -1
ERROR: request -1


In [16]:
import re

for date, df_M in data_frames_1M.items():
    if 'volume' in df_M.columns:
        df_M = df_M.iloc[df_M.volume.values > 0]
    df_M['date'] = [re.sub(' +', ' ', d) for d in df_M.date.values]
    
    input_df_S = data_frames_1s[date]
    df_S = tutils.downsample(input_df_S, '1M')
    if 'volume' in df_S.columns:
        df_S = df_S.iloc[df_S.volume.values > 0]
    
    if df_S.shape != df_M.shape:
        print('{}: Shapes disagree: {} vs. {}'.format(date, df_S.shape, df_M.shape))
    else:
        if not np.all(df_S.index.values == df_M.index.values):
            print('{}: Disagree on index values'.format(date))        
        for col in df_S.columns:
            if col == 'date':
                if not np.all(df_S.date.values == df_M.date.values):
                    print('{}: Disagree on column {}'.format(date, col))
            else:
                if not np.all(np.isclose(df_S[col].values, df_M[col].values)):
                    print('{}: Disagree on column {}'.format(date, col))

20200315: Shapes disagree: (15, 5) vs. (16, 5)
20200316: Shapes disagree: (774, 5) vs. (777, 5)
20200317: Shapes disagree: (1279, 5) vs. (1287, 5)
20200318: Shapes disagree: (867, 5) vs. (872, 5)
20200320: Disagree on column high
20200322: Shapes disagree: (264, 5) vs. (266, 5)
20200324: Shapes disagree: (1257, 5) vs. (1264, 5)
20200326: Disagree on column high
20200326: Disagree on column low
20200327: Disagree on column low
20200330: Shapes disagree: (1365, 5) vs. (1364, 5)
20200331: Shapes disagree: (1365, 5) vs. (1364, 5)
20200401: Disagree on column low
20200402: Shapes disagree: (1365, 5) vs. (1362, 5)
20200403: Disagree on column low
20200405: Disagree on column low
20200406: Disagree on column high
20200406: Disagree on column low
20200407: Shapes disagree: (1365, 5) vs. (1364, 5)
20200408: Shapes disagree: (1365, 5) vs. (1364, 5)
20200409: Disagree on column high
20200409: Disagree on column low
20200412: Disagree on column high
20200413: Shapes disagree: (1365, 5) vs. (1362, 

In [83]:
date = '20200320'
input_df = data_frames_1s[date]
df_M = data_frames_1M[date]

if 'volume' in df_M.columns:
    df_M = df_M.iloc[df_M.volume.values > 0]
df_M['date'] = [re.sub(' +', ' ', d) for d in df_M.date.values]

input_df_S = data_frames_1s[date]
df_S = tutils.downsample(input_df_S, '1M')
if 'volume' in df_S.columns:
    df_S = df_S.iloc[df_S.volume.values > 0]


In [87]:
np.where(df_S.high.values != df_M.high.values)
[df_S.iloc[195], df_M.iloc[195]]

[close              2435.75
 date     20200320 03:15:00
 high               2436.25
 low                2433.25
 open               2436.25
 Name: 1584688500.0, dtype: object, close              2435.75
 date     20200320 03:15:00
 high                2436.5
 low                2433.25
 open               2436.25
 Name: 1584688500.0, dtype: object]

ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt


ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1


ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usopt


ERROR: request -1
ERROR: request -1


...

ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1


ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt


ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1


ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usopt


ERROR: request -1
ERROR: request -1


In [6]:
# Create all of the time series request objects at the same time
tickers = [ 'ESM0']
frequency = '1s'
data_types = ['TRADES', 'BID', 'ASK']
start_dates = ['20200417 00:00:00']
end_dates = ['20200418 00:00:00']
duration = ''

reqObjList = []
for ticker in tickers:
    contractList = [ct_app.get_contract(ticker)]
    for data_type in data_types:
        for j in range(len(start_dates)):
            _single_req_obj = md_app.create_historical_data_request(contractList, is_snapshot=True, 
                                            frequency=frequency, use_rth=False, data_type=data_type,
                                            start=start_dates[j], end=end_dates[j], duration=duration)
            reqObjList.append(_single_req_obj[0])

In [427]:
min_gap = 300
intraday_gaps = []
for tws_date, df in data_frames.items():
    date = helper.convert_tws_date_to_datetime(tws_date)
    gaps = find_gaps_in_data(df, date)
    intraday_gaps.append(gaps, min_gap=min_gap)

In [430]:
for j, d in enumerate(data_frames.keys()):
    date = helper.convert_tws_date_to_datetime(d)
    ct = [len(x) for x in intraday_gaps]
    if ct[j] > 0:
        print([d, date.weekday(), ct[j], j])

['20200315', 6, 62, 0]
['20200316', 0, 79, 1]
['20200317', 1, 14, 2]
['20200318', 2, 97, 3]
['20200322', 6, 9, 6]
['20200324', 1, 15, 8]
['20200330', 0, 1, 13]
['20200409', 3, 1, 22]


In [432]:
intraday_gaps[0]

[(171.0,
  999,
  datetime.datetime(2020, 3, 15, 18, 40, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  datetime.datetime(2020, 3, 15, 18, 42, 51, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)),
 (159.0,
  1025,
  datetime.datetime(2020, 3, 15, 18, 54, 44, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  datetime.datetime(2020, 3, 15, 18, 57, 23, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)),
 (146.0,
  1040,
  datetime.datetime(2020, 3, 15, 19, 2, 17, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  datetime.datetime(2020, 3, 15, 19, 4, 43, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)),
 (102.0,
  1046,
  datetime.datetime(2020, 3, 15, 19, 7, 41, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  datetime.datetime(2020, 3, 15, 19, 9, 23, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)),
 (154.0,
  1049,
  datetime.datetime(2020, 3, 15, 19, 9, 36, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 D

In [348]:
intraday_gaps = []
for tws_date, df in data_frames.items():
    date = helper.convert_tws_date_to_datetime(tws_date)
    
    utc_datetimes = [tutils.convert_tws_date_to_utc_datetime(d) for d in df.date.values]
    utc_timestamps = [dt.timestamp() for dt in utc_datetimes]
    df.index = pd.Index(utc_timestamps, name='utc_timestamp')
    
    _, timestamp_intervals = get_trading_intervals(date)
    if not timestamp_intervals:
        print('{} should be empty'.format(tws_date))
    else:
        if min(df.index.values) < timestamp_intervals[0][0]:
            print('Data precedes start for {}'.format(tws_date))
        if max(df.index.values) > timestamp_intervals[-1][1]:
            print('Data follows end for {}'.format(tws_date))            
    

In [392]:
# Remove extra rows and resave the data frame
ticker = 'ESM0'
frequency = '1s'
for tws_date, df in data_frames.items():
    date = helper.convert_tws_date_to_datetime(tws_date)
    if date.weekday() == 6:
        utc_datetimes = [tutils.convert_tws_date_to_utc_datetime(d) for d in df.date.values]
        utc_timestamps = [dt.timestamp() for dt in utc_datetimes]
        df.index = pd.Index(utc_timestamps, name='utc_timestamp')

        # Get the trading hours
        _, timestamp_intervals = get_trading_intervals(date)

        # Save the data within the start/end dates
        new_df = df.iloc[ (df.index.values >= timestamp_intervals[0][0]) & \
                          (df.index.values <= timestamp_intervals[-1][1])]
        tutils.set_timestamp_index(new_df)        
        filename = tutils.save_daily_ib_data(new_df, ticker, frequency=frequency, 
                                             tws_date=tws_date, data_type=data_type)
        print(filename)
        print( (df.shape[0], new_df.shape[0]))

data/ESM0_1s_20200315_BID.csv
(2114, 846)
data/ESM0_1s_20200322_BID.csv
(14850, 13425)
data/ESM0_1s_20200329_BID.csv
(17680, 16680)
data/ESM0_1s_20200405_BID.csv
(16120, 15381)
data/ESM0_1s_20200412_BID.csv
(14836, 14259)


In [ ]:
########################################################################
# Compare tick data to 1s bars

# First, create a historical time series request
ticker = 'ESM0'
frequency = '1s'
data_type = 'TRADES'
start = '20200422 08:30:00'
end = '20200422 09:00:00'
duration = ''

contractList = [ct_app.get_contract(ticker)]
histReqObjList = md_app.create_historical_data_request(contractList, is_snapshot=True, frequency=frequency, 
                                        use_rth=False, data_type=data_type,
                                        start=start, end=end, duration=duration)

# Place the time series request
histReqObj = histReqObjList[0]
histReqObj.place_request()

# Get a data frame with the historical data
df_hist = pd.DataFrame.from_dict(histReqObj.get_data()[0])
df_hist = df_hist[df_hist['volume'] != 0]
df_hist.reset_index(inplace=True)

datetimes = pd.DatetimeIndex(df_hist.date.values).to_pydatetime()
timestamps = [int(d.timestamp()) for d in datetimes]
df_hist['time'] = timestamps
df_hist.set_index('time', inplace=True)
df_hist = df_hist.drop('date', axis=1)

In [ ]:
# Create a time series request
ticker = 'ESM0'
data_type = 'Last'
number_of_ticks = 0

contractList = [ct_app.get_contract(ticker)]
tickReqObjList = md_app.create_streaming_tick_data_request(contractList, data_type=data_type,
                                                    number_of_ticks=number_of_ticks)
tickReqObj = tickReqObjList[0]
tickReqObj.place_request()

cols = ['time', 'price', 'size']
prices = [{c: d[c] for c in cols} for d in tickReqObj.get_data()]
df = pd.DataFrame.from_dict(prices)
df.set_index('time', inplace=True)
ts = tutils.convert_tick_to_time_bars(df)
ts.index = ts.index - 6 * 3600

t_0, t_F = df_hist.index.values[0], df_hist.index.values[-1]
df_tick = ts[(t_0 <= ts.index.values) & (ts.index.values <= t_F)]

In [ ]:
# Compare the aggregated tick data to the 1s bars from TWS
for col in df_hist.columns:
    if not np.all(np.isclose(df_tick[col].values, df_hist[col].values)):
        print('Values for {} do not agree.'.format(col))